<a href="https://colab.research.google.com/github/ag826/AIPI590_XAI_F25/blob/main/Assignemnt_AI_Alignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment - AI Alignment
**Adil Keku Gazder**

**ag825@duke.edu**

**For AIPI 590 - XAI, Fall 2025**

**Duke University**







- Analysis of Baseline Policy
  - The provided baseline cleaning policy is very rudimentary. Although it technically does the job (of clearing all the dirt) it is not even close to being efficient (completion in >200 steps). The protocol it follows (code in the appendix) is designed such that if it is on a tile that has dirt it then cleans it, otherwise it randomly moves to another tile in one of the four directions. With this method, there are cases observed where the robot may revisit empty cells, move away from cells with dirt and waste steps. There is no local or global optimization that the robot is working towards, hence the overall total number of steps taken (which our objective is to minimize) increases. The complexity is O(n^2) where n is the size of the grid, and technically each decision is independent, so the robot is not learning from its past steps.
  - This strategy also highlights a bunch of alignment issues. Since there is no global optimization, it is not working towards maximizing the reward. Additionally, due to the lack of constraint of solving within 200 steps, there were several instances where the robot overshot this limit and failed. This highlights the dire consequences of poor alignment between reward and goal completion.
- Your Solution
  - My first priority was to replace the baseline’s random movement with a deterministic goal-based strategy which is built on the Breadth First Search (BFS) algorithm along with a nearest-dirt selection. We know that we are working with an 8x8 grid with 20 dirt cells and no obstacles and designed the solution accordingly. At every given step, the model computes the Manhattan distance (shortest distance when only moving in right angles in unit distance) and then the shortest path to the tile using a BFS search. After the robot reaches the dirt cell and cleans it, it then reruns this plan and computes a new tile.
  - The code implemented has been appended to this pdf (Code 2: Improved Policy)
  - With this solution, we guarantee that every movement that the robot takes is with a purpose. Calculating the nearest tile ensures that the robot minimizes the total distance travelled (number of steps) and BFS gives us the shortest way of getting there. We want to void unnecessary detour steps, backtracking (revisiting the same cell) or entering a situation where the robot gets stuck between two equally feasible solutions. We only rerun the loop once either the current path is finished or a tile is cleaned so we try and keep it computationally simple with this.
- Alignment Discussion
  - My objective with this assignment was to minimize unwanted movements of the robot which would unnecessarily increase the total step count. This used to occur because the reward policy was not clearly stated or addressed. The baseline policy only optimizes for the given reward (which is to clean when it comes across a cell which is dirty) but it fails to optimize for the global goal (clean all dirt with minimal steps).
  - The underlying thought in designing my solution was to add structure and sequential decision making steps into the robots thought process. It not only cleans the cell but now must optimize on how it gets to the dirty cell, hence it moves from a completion task to now a navigation and completion task. By computing the Manhattan distance and then using BFS to get the shortest path, we end up achieving both of these goals. We also have a tie-breaking mechanism that’s implemented with BFS, to ensure that the robot never gets stuck with loops.
  - The only limitations that still exist are that this algorithm still is optimizing locally for the most part and hence may be slightly suboptimal compared to a fully global planner. The flip side of this argument is that it becomes very computationally expensive to execute a full global planner.
- Performance Analysis
  - Compared to the baseline policy, our current policy significantly outperforms the baseline, as measured by the total steps taken. While the baseline frequently revisits already-cleaned cells and wastes steps wandering without direction, our policy prioritizes unexplored and high-dirt-density regions, resulting in significantly fewer total movements. The metric we used to measure it is dirt cleaned per step taken (%), which in our case is 20/56, significantly higher than 20/200 (baseline policy).
  - When it comes to failure cases, our policy continues to struggle in cases where there are clusters of dirt pockets in areas further away from where the robot started. Since the policy is greedy, it may take longer (more steps) to get to the far away dirt clusters. Another case which was observed was long paths of dirt or maps with uniform dirt distribution where our advantage over the baseline policy tends to decrease.
  - Improvements to this policy could possibly include incorporating some sort of short term memory to reduce unnecessary revisits and wandering into empty zones. Additionally we could try and optimize the global planning layer, building on the BFS method could help improve this.

In [2]:
# Code 1: Baseline Policy
def cleaning_policy(robot_pos, dirt_map):
  """
    Write your cleaning policy here!

    This example is very simple. It cleans if on dirt, otherwise it moves randomly.

    Args:
        robot_pos: [row, col] - Current position of the robot
        dirt_map: 2D numpy array where 1 means dirt

    Returns:
        action: Integer 0-4 where:
            0: Move Up
            1: Move Right
            2: Move Down
            3: Move Left
            4: Clean
    """
    # If on dirt, clean it
  if dirt_map[tuple(robot_pos)] == 1:
    return 4

  # Otherwise, move randomly
  return np.random.randint(0, 4) # Random direction

In [ ]:
# Code 2: Improved Policy

import numpy as np
from collections import deque

directions = [(-1,0,0),(0,1,1),(1,0,2),(0,-1,3)]
planned_path = []
dirt_order = []

def bfs_path(start, goal, grid_size=(8,8)):
  queue = deque()
  queue.append((start, []))
  visited = set()
  visited.add(tuple(start))
  while queue:
    pos, path = queue.popleft()
    if tuple(pos) == tuple(goal):
        return path
    for dr, dc, action in directions:
      nr, nc = pos[0]+dr, pos[1]+dc
      if 0<=nr<grid_size[0] and 0<=nc<grid_size[1] and (nr,nc) not in visited:
        visited.add((nr,nc))
        queue.append(((nr,nc), path+[action]))
    return []

def cleaning_policy(robot_pos, dirt_map):
  global planned_path, dirt_order

  # Clean if on dirt
  if dirt_map[tuple(robot_pos)] == 1:
    planned_path = []
    return 4

    # Identify remaining dirt tiles
    dirt_positions = np.argwhere(dirt_map==1)
    if len(dirt_positions) == 0:
      return 0 # no dirt left

    # Plan path if none exists
    if not planned_path:
      # Compute Manhattan distances from robot to all dirt
      distances = [abs(robot_pos[0]-d[0])+abs(robot_pos[1]-d[1]) + abs(robot_pos[1]-d[1]) for d in dirt_positions]
      # Select nearest dirt
      nearest_dirt = dirt_positions[np.argmin([abs(robot_pos[0]-d[0])+abs(robot_pos[1]-d[1]) for d in dirt_positions])]
      # Compute shortest path using BFS
      planned_path = bfs_path(robot_pos, nearest_dirt)

    # Execute next move
    if planned_path:
        return planned_path.pop(0)
    return 0

    # Code partially generated with help from Gemini 2.5 Pro and refined with ChatGPT on Friday 11/14 at 3pm